# v1 : 자기지도학습 방식  

In [ ]:
import pandas as pd
import requests
import re
from langchain_text_splitters import RecursiveCharacterTextSplitter

## 언급되는 조문 스캔 (테스트)

In [ ]:
def fetch_precedent_content(url : str):
    try :
        response = requests.get(url)
        response.raise_for_status()
        json = response.json()
        if not isinstance(json , dict) or not 'PrecService' in json.keys():
            print(f"{url} : the json file can't be found") # Json 파일이 제공되지 않는 경우
            return
        return response.json().get('PrecService', "")
        
    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL {url} : {e}")
        return
    except Exception as e:
        print(f"An unexpected error occurred while fetching the URL ({url}): {e}")
        return

# 받아온 json 파일에 대해 언급되는 조문 스캔
def scan_ref(content : dict, law_mapper : dict[str, int]):
    law_pattern = '|'.join(re.escape(law) for law in law_mapper.keys())
    ref = []
    for part in content.keys():
        text = content[part]
        pattern = rf'({law_pattern})\s제(\d+)조(?:\s제(\d+)항)?(?:\s제(\d+)호)?' # -법 -조 (-항)
        matches =  re.findall(pattern, text)

        if len(matches) > 0:
            for m in matches:
                law = m[0] or 0
                article = m[1] or 0
                paragraph = m[2] or 0
                subparagraph = m[3] or 0
                data = f"{law_mapper.get(law)}-{article}-{paragraph}-{subparagraph}"
                ref.append(data)
    return ref

In [ ]:
url = "https://www.law.go.kr/DRF/lawService.do?OC=fehs0611&target=prec&ID=166740&type=JSON&mobileYn="
content = fetch_precedent_content(url)

content

{'판시사항': '',
 '참조판례': '',
 '사건종류명': '형사',
 '판결요지': '',
 '참조조문': '',
 '선고일자': '20120615',
 '법원명': '서울고등법원',
 '사건명': '강간상해·절도',
 '판례내용': '【피 고 인】 <br/>【항 소 인】 쌍방<br/>【검    사】 유정현(기소), 박문수(공판)<br/>【변 호 인】 변호사 김광순(국선)<br/>【원심판결】 인천지방법원 부천지원 2012. 2. 10. 선고 2011고합138 판결<br/>【주    문】<br/>원심판결을 파기한다.  <br/>피고인을 징역 5년에 처한다. <br/>피고인에 대한 정보를 7년간 공개 및 고지한다(다만, 성범죄의 요지는 판시 제1항 기재 범죄에 한한다).<br/><br/>【이    유】1. 항소이유의 요지<br/>가. 피고인<br/>    (1) 사실오인 또는 법리오해 (판시 강간상해죄에 대하여)<br/>    피고인은 피해자를 강간한 사실이 없다. 피고인의 모발이 발견된 승용차 보닛이 강간 범행이 일어난 장소라고 단정할 수 없고, 경찰관이 위 모발을 채취함에 있어 ‘경기도지방경찰청 현장감식실시규칙’ 등을 준수하지 않았기 때문에 그와 관련된 증거는 위법수집증거에 해당하여 증거능력이 없으며, 피고인을 범인으로 지목한 피해자의 진술은 범인식별절차에서 요구되는 절차적 요건을 갖추었다고 볼 수 없어 그 신빙성이 없다. 당시 피고인은 교통사고로 인한 후유증 등으로 피해자를 어깨에 메고 111.7m나 갈 수 있는 신체상태도 아니었다. 그럼에도 판시 강간상해죄를 유죄로 인정한 원심판결에는 사실오인 또는 법리오해의 위법이 있다. <br/>    (2) 양형부당 <br/>    원심의 형(징역 4년)은 너무 무거워서 부당하다.<br/>나. 검사<br/>    (1) 사실오인 (이유무죄 부분)<br/>    피해자가 입은 상해의 부위와 정도, 피해자를 검진한 의사공소외 1의 소견 등을 종합하면, 피고인의 강간범행이 기수에 이른 사실이 인정됨에도, 강간미수의 

In [ ]:
url = "https://www.law.go.kr/DRF/lawService.do?OC=fehs0611&target=prec&ID=166740&type=JSON&mobileYn="
content = fetch_precedent_content(url)


# ["특강법", "형법", "형사소송법", "성폭력범죄의 처벌 등에 관한 특례법"]
# 띄어쓰기 및 맞춤법 유의
mapper = {
    "형법" : 0,
    "형사소송법" : 1,
    "성폭력범죄의 처벌 등에 관한 특례법" : 2,
}


ref = scan_ref(content, mapper)
ref

['0-35-0-0',
 '0-35-0-0',
 '0-37-0-0',
 '1-364-2-0',
 '1-369-0-0',
 '0-301-0-0',
 '0-329-0-0',
 '0-42-0-0',
 '0-37-0-0',
 '0-42-0-0',
 '0-53-0-0',
 '2-37-1-1',
 '1-325-0-0']

## Query - Pos 쌍 생성
query : 법령  
pos : (연관) 판례문 내용 문장

In [ ]:
url = "https://www.law.go.kr/DRF/lawService.do?OC=fehs0611&target=prec&ID=166740&type=JSON&mobileYn="
content = fetch_content(url)

doc = precedent_doc_formatting(content)

print(doc)

extract_sentences_from_format(doc)

{'판시사항': '', '판결요지': '', '사건명': '강간상해·절도', '판례내용': {'피고인': [], '항소인': ['쌍방'], '검사': ['유정현(기소), 박문수(공판)'], '변호인': ['변호사 김광순(국선)'], '원심판결': ['인천지방법원 부천지원 2012. 2. 10. 선고 2011고합138 판결'], '주문': ['원심판결을 파기한다.', '피고인을 징역 5년에 처한다.', '피고인에 대한 정보를 7년간 공개 및 고지한다(다만, 성범죄의 요지는 판시 제1항 기재 범죄에 한한다).'], '이유': ['항소이유의 요지', '피고인', '사실오인 또는 법리오해 (판시 강간상해죄에 대하여)', '피고인은 피해자를 강간한 사실이 없다. 피고인의 모발이 발견된 승용차 보닛이 강간 범행이 일어난 장소라고 단정할 수 없고', '경찰관이 위 모발을 채취함에 있어 ‘경기도지방경찰청 현장감식실시규칙’ 등을 준수하지 않았기 때문에 그와 관련된 증거는 위법수집증거에 해당하여 증거능력이 없으며', '피고인을 범인으로 지목한 피해자의 진술은 범인식별절차에서 요구되는 절차적 요건을 갖추었다고 볼 수 없어 그 신빙성이 없다. 당시 피고인은 교통사고로 인한 후유증 등으로 피해자를 어깨에 메고 111.7m나 갈 수 있는 신체상태도 아니었다. 그럼에도 판시 강간상해죄를 유죄로 인정한 원심판결에는 사실오인 또는 법리오해의 위법이 있다.', '양형부당', '원심의 형(징역 4년)은 너무 무거워서 부당하다.', '검사', '사실오인 (이유무죄 부분)', '피해자가 입은 상해의 부위와 정도, 피해자를 검진한 의사공소외 1의 소견 등을 종합하면, 피고인의 강간범행이 기수에 이른 사실이 인정됨에도', '강간미수의 성립만을 인정한 원심판결에는 사실오인의 위법이 있다.', '양형부당', '원심의 형은 너무 가벼워서 부당하다.', '판단', '직권 판단', '원심이 적법하게 채택하여 조사한 증거들에 의하면', '피고인은 2001. 5. 25. 서울지방법원 남부지원에서 강제추행치상죄로 

['강간상해·절도',
 '쌍방',
 '유정현(기소), 박문수(공판)',
 '변호사 김광순(국선)',
 '인천지방법원 부천지원 2012. 2. 10. 선고 2011고합138 판결',
 '원심판결을 파기한다.',
 '피고인을 징역 5년에 처한다.',
 '피고인에 대한 정보를 7년간 공개 및 고지한다(다만, 성범죄의 요지는 판시 제1항 기재 범죄에 한한다).',
 '항소이유의 요지',
 '피고인',
 '사실오인 또는 법리오해 (판시 강간상해죄에 대하여)',
 '피고인은 피해자를 강간한 사실이 없다. 피고인의 모발이 발견된 승용차 보닛이 강간 범행이 일어난 장소라고 단정할 수 없고',
 '경찰관이 위 모발을 채취함에 있어 ‘경기도지방경찰청 현장감식실시규칙’ 등을 준수하지 않았기 때문에 그와 관련된 증거는 위법수집증거에 해당하여 증거능력이 없으며',
 '피고인을 범인으로 지목한 피해자의 진술은 범인식별절차에서 요구되는 절차적 요건을 갖추었다고 볼 수 없어 그 신빙성이 없다. 당시 피고인은 교통사고로 인한 후유증 등으로 피해자를 어깨에 메고 111.7m나 갈 수 있는 신체상태도 아니었다. 그럼에도 판시 강간상해죄를 유죄로 인정한 원심판결에는 사실오인 또는 법리오해의 위법이 있다.',
 '양형부당',
 '원심의 형(징역 4년)은 너무 무거워서 부당하다.',
 '검사',
 '사실오인 (이유무죄 부분)',
 '피해자가 입은 상해의 부위와 정도, 피해자를 검진한 의사공소외 1의 소견 등을 종합하면, 피고인의 강간범행이 기수에 이른 사실이 인정됨에도',
 '강간미수의 성립만을 인정한 원심판결에는 사실오인의 위법이 있다.',
 '양형부당',
 '원심의 형은 너무 가벼워서 부당하다.',
 '판단',
 '직권 판단',
 '원심이 적법하게 채택하여 조사한 증거들에 의하면',
 '피고인은 2001. 5. 25. 서울지방법원 남부지원에서 강제추행치상죄로 징역 2년 6월에 집행유예 3년을 선고받고',
 '그 유예기간 중인 2002. 3. 22. 서울지방법원에서 강간치상죄로 징역 3년을 선고받

In [37]:
import os

keywords = [
    "강간",
    # "강도",
    # "사기",
    # "살인",
    # "절도",
    # "추행",
    # "폭행",
]

##
law_mapper = {
    "형법" : 0,
    "형사소송법" : 1,
    "성폭력범죄의 처벌 등에 관한 특례법" : 2,
}
##

def _load_law_docs_as_list(mapper : dict[str, int], dir_path : str) -> list[list[tuple[str, str]]]: 
    # laws = [[] for _ in range(len(mapper.keys()))]
    laws = []

    for k in mapper.keys():
        try:
            fpath = os.path.join(dir_path, k)
            df = pd.read_excel(f"{fpath}.xlsx")
        except FileNotFoundError as e:
            print(f"The file {fpath} does not exist")
            raise e

        for i, row in df.iterrows():
            if row['preamble'] == 1 : continue
            numbering = mapper.get(k)
            data = (f"{numbering}-{row['article']}-{row['paragraph']}-{row['subparagraph']}", row['content'])
            # laws[numbering].append(data)
            laws.append(data)
            
    return laws

def _load_precedent_docs_as_dataframe(keyword : str, dir_path : str) -> pd.DataFrame:
    try:
        fpath = os.path.join(dir_path, f"판례_{keyword}.xlsx")
        return pd.read_excel(fpath).fillna("")
    except FileNotFoundError as e:
        print(f"The file {fpath} does not exist")
        raise(e)

def id_to_law_text(id : str, laws : list[list[tuple[str, str]]]):
    for l in laws: # list
        for t in l : # tuple
            if t[0] == id:
                return t[1]
            
    return None

def law_text_to_id(text: str, laws : list[list[tuple[str, str]]]):
    for l in laws: # list
        for t in l : # tuple
            if t[1] == text:
                return t[0]
            
    return None

In [34]:
_load_precedent_docs_as_dataframe("강간", "../fetched_data").head()

,판시사항,참조판례,사건종류명,판결요지,참조조문,선고일자,법원명,사건명,판례내용,사건번호,사건종류코드,판례정보일련번호,선고,판결유형,법원종류코드
0,,,형사,,,20240621,대전고등법원,성폭력범죄의처벌등에관한특례법위반(강간등치상),【피 고 인】 피고인<br/>【항 소 인】 쌍방<br/>【검 사】 이준태(기소...,2024노156,400102,599839,선고,판결,400202
1,,,형사,,,20240313,대전지방법원 논산지원,성폭력범죄의처벌등에관한특례법위반(강간등치상),"【피 고 인】 피고인<br/>【검 사】 이준태(기소), 한경우(공판)<br/>...",2023고합61,400102,599837,선고,판결,400202
2,,,형사,,,20231011,부산고등법원(창원),성폭력범죄의처벌등에관한특례법위반(친족관계에의한강간)·아동복지법위반(아동학대)·아동복...,【피 고 인】 피고인 1<br/>【피고인 겸 피부착명령청구자】 피고인 겸 피부착명령...,"2023노130, 2023전노14(병합), 15(병합), 16(병합)",400102,240625,선고,판결,400202
3,,,형사,,,20230907,서울고등법원,성폭력범죄의처벌및피해자보호등에관한법률위반(특수준강간),【피 고 인】 피고인 1 외 2인<br/>【항 소 인】 검사<br/>【검 사】...,2022노1982,400102,240761,선고,판결,400202
4,,,형사,,,20230726,광주고등법원(제주),상해(인정된죄명:강간상해),【피 고 인】 피고인<br/>【항 소 인】 피고인 및 검사<br/>【검 사】 ...,2023노17,400102,240113,선고,판결,400202


In [ ]:
from FlagEmbedding import FlagModel
import numpy as np
import torch

def generate_query_pos_pair(
    model: FlagModel,
    keywords: list[str],
    law_mapper: dict[int, str],
    dir_path: str,
    similarity_threshold: float,
):
    laws = _load_law_docs_as_list(law_mapper, dir_path)
    laws_without_id = [l[1] for l in laws]
    # laws_id = [l[0] for l in laws]

    queries_embeddings = model.encode_queries(laws_without_id)

    # list of {'query' : str, "pos" : list[str]}
    dataset = []

    pairs = [
        {
            'query': laws_without_id[i],
            'pos': []
        }
        for i in range(len(laws_without_id))
    ]

    dataset.extend(pairs)

    for k in keywords:  # 범죄유형
        file = _load_precedent_docs_as_dataframe(k, dir_path)
        
        # file row data 하나 당 문서 한 개에 해당
        for _, row in file.iterrows():
            content = row.to_dict()

            formatted_content = content_formatting(content)
            sentences = extract_sentences_from_format(formatted_content)
            corpus_embeddings = model.encode_corpus(sentences)            

            # numpy -> torch (GPU로 이동)
            emb_q = torch.tensor(queries_embeddings, device="cuda", dtype=torch.float32)
            emb_c = torch.tensor(corpus_embeddings, device="cuda", dtype=torch.float32)

            # similarity 계산 (GPU 상에서 연산)
            similarity = torch.matmul(emb_q, emb_c.T)

            # mask 계산 및 CPU로 복귀
            mask = similarity >= similarity_threshold
            mask = mask.cpu().numpy()

            # similarity = queries_embeddings @ corpus_embeddings.T
            # mask = similarity >= similarity_threshold

            for i in range(mask.shape[0]):
                pos_list = dataset[i].get('pos')
                pos_list.extend([sentences[j] for j in np.where(mask[i])[0]])

    return dataset

ModuleNotFoundError: No module named 'FlagEmbedding'

In [ ]:
model = FlagModel('BAAI/bge-m3', use_fp16=True)

keywords = [
    "강간",
    # "강도",
    # "사기",
    # "살인",
    # "절도",
    # "추행",
    # "폭행",
]

query_mapper = {
    "형법" : 0,
    "형사소송법" : 1,
    "성폭력범죄의 처벌 등에 관한 특례법" : 2,
}

In [ ]:
dataset = generate_query_pos_pair(model, keywords, query_mapper, "/content/drive/My Drive/Colab Notebooks/data", 0.85)

In [ ]:
import json

with open("ft_data2.json", "w", encoding="utf-8") as f:
    json.dump(dataset, f, ensure_ascii=False, indent=4)

In [1]:
import json

with open("../data/ft_data2.json", "r", encoding="utf-8") as f:
    ft_data2 = json.load(f)

In [2]:
import pandas as pd

ft_data2_df = pd.DataFrame(ft_data2)

In [3]:
print(len(dict(ft_data2_df['query'])))
print(len(ft_data2_df))


2843
2843


In [ ]:
ft_data2_sample = ft_data2_df.loc[ft_data2_df['pos'].apply(lambda x : len(x) > 0)]

# ft_data2_sample.to_excel("../test_output/ft_data2_samples.xlsx")

ft_data2_obj = []
for i, row in ft_data2_sample.iterrows():
    tmp = {
        'query' : row['query'],
        'pos' : row['pos'],
    }
    ft_data2_obj.append(tmp)


with open("../test_output/ft_data2_samples.json", 'w', encoding='utf-8') as f:
    json.dump(ft_data2_obj, f, ensure_ascii=False, indent=4)


## 기타 테스트 코드

In [20]:
law_mapper = {
    "형법" : 0,
    "형사소송법" : 1,
    "성폭력범죄의 처벌 등에 관한 특례법" : 2,
}

test = _load_law_docs_as_list(law_mapper, "../fetched_data")

for i, t in enumerate(test):
    print(f"== {list(law_mapper.keys())[i]} ==")
    for e in t:
        print(e)

NameError: name '_load_law_docs_as_list' is not defined

In [ ]:
laws = _load_law_documents_as_list(law_mapper, "../fetched_data")

print(id_to_law_text("0-5-0-1", laws))
print(law_text_to_id("내란의 죄", laws))

내란의 죄
0-5-0-1
